### 手写朴素贝叶斯分类器

本文件主要是根据朴素贝叶斯理论部分，通过Python实现朴素贝叶斯分类器，并通过创建数据进行测试

数据说明：

- 数据是自己创建的
- 标签数据，1 代表侮辱性文字， 0 代表正常言语


#### 具体步骤如下：

- 词向量构建；
- 相关概率计算；
- 平滑处理；
- 朴素贝叶斯分类函数；
- 测试

#### 一、词向量构建

In [1]:
# 创建一些实验样本
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 1 代表侮辱性文字， 0 代表正常言语
    return postingList, classVec  # 返回词条集合和类别标签


# 创建一个在所有文档中出现的不重复词的列表
# 使用了python的set数据类型，将词条列表输给set构造函数，set就会返回一个不重复的词表
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空集
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 创建两个集合的并集
    return list(vocabSet)


# 文档向量
def setOfWords2Vec(vocabList, inputSet):  # 词汇表、某个文档
    returnVec = [0] * len(vocabList)  # 创建一个和词汇表等长的0向量
    for word in inputSet:  # 遍历文档所有单词
        if word in vocabList:  # 判断，如果出现了词汇表中的单词
            returnVec[vocabList.index(word)] = 1  # 文档向量对应位置设为1
        else:
            print("the word: %s is not in my Vocabulary!" % word)  # 否则输出不再我的词汇表中
    return returnVec  # 输出是文档向量，向量的每个元素为1或0，分别表示词汇表中的单词在输入文档中是否出现


# 测试
listOPosts, listClasses = loadDataSet()
print("训练样本：", listOPosts)
print("样本标签：", listClasses)
myVocabList = createVocabList(listOPosts)
print("我的词列表为：", myVocabList)

print("第一个样本的词向量为:", setOfWords2Vec(myVocabList, listOPosts[0]))
print("第二个样本的词向量为:", setOfWords2Vec(myVocabList, listOPosts[1]))

训练样本： [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
样本标签： [0, 1, 0, 1, 0, 1]
我的词列表为： ['quit', 'licks', 'has', 'my', 'flea', 'dalmation', 'mr', 'food', 'love', 'is', 'help', 'problems', 'so', 'garbage', 'him', 'to', 'stupid', 'stop', 'worthless', 'please', 'maybe', 'how', 'posting', 'ate', 'take', 'I', 'steak', 'not', 'park', 'buying', 'cute', 'dog']
第一个样本的词向量为: [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
第二个样本的词向量为: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]


#### 二、相关概率计算

我们得到了每个样本的词向量，如下图：（由于set函数每次输出不一样所以这里得到可能不一样，但结构是一样的）

![](1.png)

看到这些数据，是不是似曾相识？是的，这跟理论部分的西瓜数据类似，只不过这里的每个特征是一个单词，每个单词的属性值只有1和0，1表示该单词在该文本中出现，0则相反。

> 首先，计算先验概率：
> $$p(侮辱类) = \frac{3}{6} = 0.5，p(非侮辱类)=\frac{3}{6}=0.5$$
> 再求，条件概率，（用出现次数除以相应类的总词数）例如：
> $$p(第一个词|侮辱类) = \frac{2}{19} \approx 0.10526$$
> $$p(第一个词|非侮辱类) = \frac{1}{24} \approx 0.04167$$
> 注：这边与西瓜数据集的计算方法有些不同

下面我们直接来看代码，注意：这里面使用了Numpy进行计算处理，大大方便了计算。


In [5]:
from numpy import *

# 朴素贝叶斯分类器训练函数
def trainNB01(trainMatrix, trainCategory):  # 文档矩阵、由每篇文档类别标签所构成的向量
    numTrainDocs = len(trainMatrix)  # 计算训练样本个数
    numWords = len(trainMatrix[0])  # 每个样本的词向量中的元素个数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 计算先验概率

    p0Num = zeros(numWords)  # 创建一个0填充的矩阵，并且元素个数与词向量的元素个数相同，用于记录正常词语
    p1Num = zeros(numWords)  # 用于记录侮辱性词语

    p0Denom = 0.0
    p1Denom = 0.0
        
    for i in range(numTrainDocs):  # 遍历训练集中的所有文档
        if trainCategory[i] == 1:  # 如果类标签为1（即侮辱性文档）
            p1Num += trainMatrix[i]  # 该词对应的个数对应+1
            p1Denom += sum(trainMatrix[i])  # 加上该文档的总词数
        else:  # 如果类标签为0，做同样操作
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num / p1Denom  # 对每个元素除以该类别中的总词数
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive  # 返回两个向量一个概率


# 测试
listOPosts, listClasses = loadDataSet()
print("训练样本：", listOPosts)
print("样本标签：", listClasses)
myVocabList = createVocabList(listOPosts)
print("我的词列表为：", myVocabList)  # 得到不含重复词的词列表

trainMat = []
# 使用词向量来填充trainMat列表
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

# 打印trainMat
for i in range(len(trainMat)):
    print("第 %d 个样本的词向量为：" % (i + 1), trainMat[i])

p0V, p1V, pAb = trainNB01(trainMat, listClasses)
print("任意文档属于侮辱性文档的概率为：", pAb)
print("词列表中每个词在类别0中条件概率：", p0V)
print("词列表中每个词在类别1中条件概率：", p1V)

训练样本： [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
样本标签： [0, 1, 0, 1, 0, 1]
我的词列表为： ['quit', 'licks', 'has', 'my', 'flea', 'dalmation', 'mr', 'food', 'love', 'is', 'help', 'problems', 'so', 'garbage', 'him', 'to', 'stupid', 'stop', 'worthless', 'please', 'maybe', 'how', 'posting', 'ate', 'take', 'I', 'steak', 'not', 'park', 'buying', 'cute', 'dog']
第 1 个样本的词向量为： [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
第 2 个样本的词向量为： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]
第 3 个样本的词向量为： [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1

#### 三、平滑处理

- 利用贝叶斯分类器进行文档分类时，要计算多个概率的乘积以获得文档属于某个类的概率，计算：$p(w_1|1)p(w_2|1)p(w_3|1)...$

- 如果某些词在某一类中未出现，那么概率就会为0，在计算多个概率相乘以获得文档属于某个类别的概率的时候，值就会为0，为了降低这种影响，我们可以将所有词的出现次数初始化为1，并将分母初始化为2.0。


    p0Num = ones(numWords)
    p1Num = ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0

- 另一个问题是下溢出，由于太多很小的数相乘造成的，当计算：$p(w_1|1)p(w_2|1)p(w_3|1)...$时，大部分因子比较小，所以程序会下溢出，可以对其求对数，将连乘变为连加，虽然得到的值不同，但是由于对数函数不会影响原函数的趋势，以及极大值等，相当于只是纵轴上做了缩放，所以不会影响最终的结果


    p1Vect = log(p1Num / p1Denom)
    p0Vect = log(p0Num / p0Denom)


In [6]:
# 朴素贝叶斯分类器训练函数（修改后）
def trainNB02(trainMatrix, trainCategory):  # 文档矩阵、由每篇文档类别标签所构成的向量
    numTrainDocs = len(trainMatrix)  # 计算训练样本个数
    numWords = len(trainMatrix[0])  # 每个样本的词向量中的元素个数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 计算p(1)

    p0Num = ones(numWords)  # 创建一个1填充的矩阵，并且元素个数与词向量的元素个数相同，用于记录正常词语
    p1Num = ones(numWords)  # 用于记录侮辱性词语

    p0Denom = 2.0
    p1Denom = 2.0

    for i in range(numTrainDocs):  # 遍历训练集中的所有文档
        if trainCategory[i] == 1:  # 如果类标签为1（即侮辱性文档）
            p1Num += trainMatrix[i]  # 每个侮辱性文档中每个词对应位置个数+1
            p1Denom += sum(trainMatrix[i])  # 每个侮辱性文档的总词数叠加
        else:  # 如果类标签为0，做同样操作
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)  # 对每个元素除以该类别中的总词数，并求对数
    p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive  # 返回两个向量一个概率

#### 四、朴素贝叶斯分类函数

In [7]:
# 朴素贝叶斯分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1): # 要分类的向量以及使用函数trainNB0计算的三个概率
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)  # 两个向量相乘（对应元素相乘），然后将词汇表中所有词的对应值相加，然后将该值加到类别的对数概率上
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0: # 比较类别的概率
        return 1
    else:
        return 0

#### 五、测试

In [8]:
# 测试
listOPosts, listClasses = loadDataSet()
print("训练样本：", listOPosts)
print("样本标签：", listClasses)
myVocabList = createVocabList(listOPosts)
print("我的词列表为：", myVocabList)  # 得到不含重复词的词列表

trainMat = []
# 使用词向量来填充trainMat列表
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

p0V, p1V, pAb = trainNB0(trainMat, listClasses)

testEntry = ['love', 'my', 'dalmation']
thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
print(testEntry, '分类为: ', classifyNB(thisDoc, p0V, p1V, pAb))
testEntry = ['stupid', 'garbage']
thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
print(testEntry, '分类为: ', classifyNB(thisDoc, p0V, p1V, pAb))

训练样本： [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
样本标签： [0, 1, 0, 1, 0, 1]
我的词列表为： ['quit', 'licks', 'has', 'my', 'flea', 'dalmation', 'mr', 'food', 'love', 'is', 'help', 'problems', 'so', 'garbage', 'him', 'to', 'stupid', 'stop', 'worthless', 'please', 'maybe', 'how', 'posting', 'ate', 'take', 'I', 'steak', 'not', 'park', 'buying', 'cute', 'dog']
['love', 'my', 'dalmation'] 分类为:  0
['stupid', 'garbage'] 分类为:  1
